In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob
import dataclasses
import pandas as pd

MainProcess


In [2]:
# Prepare datasets
fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr_res/')

In [3]:
files = glob.glob(fls.result_dir + '/many_abbr_res/' + '*.pickle')
dicts = []
for f in files:    
    data = fls.dill_load(f)    
    d = data.modifier_values
    d['git_commit_id'] = data.git_commit_id
    d['cv_score'] = fls.score_competition_metric(data.inferred_test_data, data.test_data)
    dicts.append(d)
p = pd.DataFrame(dicts)

In [4]:
p

,seed,scale_percentile_value,img_size,n_epochs,model_name,use_pretrained_weights,box_size,trust,fix_norm_bug,weight_decay,...,erasing,use_albumentations,confidence_threshold,include_multi_motor,tom,mba,aba,ycw,git_commit_id,cv_score
0,0,3.547847,512,41,yolov8m,False,16,1,True,0.000607,...,0.0,False,0.374857,True,True,True,False,True,a103863a276b4f124f36587b568e788ff5d70412,0.651396
1,10,4.824007,384,36,yolo11m,False,29,4,False,0.000689,...,0.4,False,0.520648,False,True,True,False,False,a103863a276b4f124f36587b568e788ff5d70412,0.245447
2,1,3.047286,576,96,yolov8m,True,18,1,False,0.000828,...,0.0,True,0.406082,False,True,True,True,True,a103863a276b4f124f36587b568e788ff5d70412,0.357417
3,2,2.046449,320,43,yolov8l,False,29,2,True,0.000188,...,0.0,False,0.452213,True,True,False,True,False,a103863a276b4f124f36587b568e788ff5d70412,0.000000
4,3,1.342597,384,38,yolov8m,True,28,0,False,0.000479,...,0.0,False,0.544692,False,False,True,True,False,a103863a276b4f124f36587b568e788ff5d70412,0.057661
5,4,4.772224,640,60,yolo11m,False,31,2,False,0.000802,...,0.4,True,0.538561,True,False,False,False,True,a103863a276b4f124f36587b568e788ff5d70412,0.020747
6,5,4.220012,320,84,yolov8l,True,24,5,False,0.000408,...,0.0,True,0.404290,True,False,True,True,False,a103863a276b4f124f36587b568e788ff5d70412,0.256917
7,6,3.152657,512,47,yolo11m,True,21,2,True,0.000674,...,0.0,True,0.374456,False,False,True,True,False,a103863a276b4f124f36587b568e788ff5d70412,0.765027
8,7,3.500382,576,91,yolov8l,True,28,0,True,0.000005,...,0.4,False,0.382042,True,False,False,True,False,a103863a276b4f124f36587b568e788ff5d70412,0.222399
9,8,2.573243,640,80,yolov8l,True,17,3,False,0.000134,...,0.4,True,0.412206,False,False,False,True,True,a103863a276b4f124f36587b568e788ff5d70412,0.270164
